In [2]:
import pandas as pd
# csv location
data_path = "australia.csv"
# read CSV dataset
data = pd.read_csv(data_path)
data.set_index('id', inplace=True)
data

,city,lat,lng,state/territory,population
id,,,,,
1,Adelaide,-34.928661,138.598633,South Australia,1145000.0
2,Adelaide River,-13.239430,131.107330,Northern Territory,237.0
3,Albany,-35.003101,117.865952,Western Australia,26445.0
4,Albury,-36.074823,146.924006,New South Wales,104258.0
5,Alice Springs,-23.697479,133.883621,Northern Territory,27710.0
...,...,...,...,...,...
215,Woomera,-31.199810,136.832581,South Australia,450.0
216,Yamba,-29.435450,153.360306,New South Wales,1806.0
217,Yeppoon,-23.126829,150.744064,Queensland,10769.0


In [68]:
a = data.index.values
a

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [33]:
import geopy.distance 
from math import inf
import heapq

In [89]:
class Node(object):
  def __init__(self, id, info):
    self.id = id
    self.info = info
    self.parent = None
    self.closed = False
    self.g = inf
    self.f = inf
        
  def calculate_real_distance(self, node):
    # a distância pela estrada é 10% maior que a distância em linha reta
    distance = self.calculate_heuristic_distance(node) * 1.1
    return distance

  def calculate_heuristic_distance(self, node):
    p1 = (self.info.lat, self.info.lng)
    p2 = (node.info.lat, node.info.lng)
    distance = geopy.distance.distance(p1, p2).km
    return distance
 

In [82]:
class Map(object):
  def __init__(self, data):
    self.grid = [None]
    for id in data.index.values:
      info = data.loc[id, ['city','lat', 'lng']]
      self.grid.append(Node(id, info))

  def get_node(self, id):
    return self.grid[id]

  def get_successors(self, id): 
    successors = []
    valid_id = data.index.values
    # id par
    if id%2 == 0:
      if (id + 2) in valid_id:
        successors.append(id + 2)
      if (id - 1) in valid_id:
        successors.append(id - 1)
      if (id - 2) in valid_id:
        successors.append(id - 2)
    # id ímpar
    else:
      if (id + 2) in valid_id:
        successors.append(id + 2)
      if (id + 1) in valid_id:
        successors.append(id + 1)
      if (id - 2) in valid_id:
        successors.append(id - 2)        
    return successors

In [167]:
class PathPlanner(object):
  def __init__(self, map):
    self.map = map

  @staticmethod
  def construct_path(goal_node):
    node = goal_node
    reversed_path = []
    
    while node is not None:
        reversed_path.append((node.info.name, node.info.city))
        
        node = node.parent
    return reversed_path[::-1]  # This syntax creates the reverse list

  def a_star(self, start_id, goal_id):

    pq = []
    node = self.map.get_node(start_id)
    goal_node = self.map.get_node(goal_id)
    node.g = 0
    node.f = node.calculate_heuristic_distance(goal_node)
    heapq.heappush(pq, (node.f, node))
    while len(pq) > 0:
        f, node = heapq.heappop(pq)
        node.closed = True
        if node == goal_node:
            return self.construct_path(goal_node), goal_node.f
        for successor in self.map.get_successors(node.id):
            node_suc = self.map.get_node(successor)
            if not node_suc.closed:
                h = node_suc.calculate_heuristic_distance(goal_node)
                cost = node.g + node.calculate_real_distance(node_suc) + h
                if node_suc.f > cost:
                    node_suc.g = node.g + node.calculate_real_distance(node_suc)
                    node_suc.f = cost
                    node_suc.parent = node
                    heapq.heappush(pq, (node_suc.f, node_suc))

In [168]:
def calculate_arestas(grid, valid_id):
  cities = grid[1:]
  arestas = []
  for city in cities:
    id = city.id
    if id%2 == 1:
      if (id + 1) in valid_id:
        arestas.append((str(id), str(id + 1), {'weight': city.calculate_real_distance(grid[id + 1])}))
    if (id + 2) in valid_id:
      arestas.append((str(id), str(id + 2), {'weight': city.calculate_real_distance(grid[id + 2])}))
    
  return arestas



In [171]:
start_id = 5
goal_id = 219

map = Map(data)
path_planner = PathPlanner(map)

path, cost = path_planner.a_star(start_id, goal_id)

print("A distância do início ao fim é ",cost)

print("\nO menor caminho é: \n\n")

for item in path:
  print(item)

A distância do início ao fim é  166286.86783760806

O menor caminho é: 


(5, 'Alice Springs')
(6, 'Andamooka')
(8, 'Armidale')
(10, 'Ayr')
(12, 'Ballarat')
(11, 'Bairnsdale East')
(13, 'Ballina')
(15, 'Batemans Bay')
(16, 'Bathurst')
(18, 'Bendigo')
(20, 'Bicheno')
(22, 'Birdsville')
(24, 'Bordertown')
(26, 'Bourke')
(28, 'Brisbane')
(30, 'Broome')
(32, 'Bundaberg')
(34, 'Burnie')
(36, 'Byron Bay')
(38, 'Cairns')
(37, 'Caboolture')
(39, 'Caloundra')
(41, 'Canberra')
(43, 'Ceduna')
(45, 'Charleville')
(47, 'Clare')
(49, 'Cobram')
(51, 'Colac')
(53, 'Cowell')
(55, 'Cranbourne')
(57, 'Dalby')
(59, 'Deniliquin')
(61, 'Dubbo')
(62, 'East Maitland')
(64, 'Eidsvold')
(66, 'Esperance')
(68, 'Forbes')
(70, 'Gawler')
(69, 'Forster')
(71, 'Geelong')
(73, 'Geraldton')
(75, 'Gladstone')
(77, 'Goondiwindi')
(79, 'Griffith')
(81, 'Gympie South')
(83, 'Hamilton')
(85, 'Hobart')
(87, 'Hughenden')
(89, 'Inverell')
(91, 'Kalbarri')
(93, 'Karratha')
(95, 'Katanning')
(97, 'Katoomba')
(99, 'Kiama')
(100, 

In [170]:
import networkx as nx 

G = nx.Graph()

valid_id = data.index.values
vertices = [str(id) for id in data.index.values]
arestas = calculate_arestas(map.grid, valid_id)

G.add_nodes_from(vertices)
G.add_edges_from(arestas)

ccm = nx.dijkstra_path(G,source = '5', target = '219')
custo_total = nx.dijkstra_path_length(G,source = '5', target = '219')

print("Validando resultado encontrado pelo algoritmo A* implementado:")
print("A distância do início ao fim calculado pelo algoritmo de Dijkstra é", custo_total)

Validando resultado encontrado pelo algoritmo A* implementado:
A distância do início ao fim calculado pelo algoritmo de Dijkstra é 166286.86783760806
